In [2]:
from IPython.display import display, HTML
display(HTML('<style>.container { width:100% !important; }</style>'))

In [3]:
import re
import numpy as np
import pandas as pd

from collections import Counter, defaultdict

In [4]:
data = pd.read_csv('spam.csv')

In [5]:
data

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will �_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [7]:
a = data.text.apply(lambda x: re.findall(r'\w+', x.lower()))

In [24]:
c = Counter(j for i in a for j in i)

In [58]:
words = np.array([i[0] for i in c.most_common(int(len(c) * 0.1))])

In [39]:
enc_sent = np.array([[word in sentence for word in words] for sentence in a])

$$p\left(C_k\right)=\frac{\text{количество примеров имещих класс } C_k}{\text{количество примеров всего}}$$

$$p(w_i|C_k)=\frac{\text{сколько раз слово встретилось в предложениях класса } C_k + 1}{\text{общее количество слов в преложениях класса } C_k + \text{число уникальных слов вообще}}$$

$$\hat{y}=\operatorname{argmax} p\left(C_k\right)\prod p\left(w_i|C_k\right)$$

In [42]:
class_prob = np.array([
    (data.type == 'ham').sum() / len(data),
    (data.type == 'spam').sum() / len(data)
])

In [44]:
total_words = np.array([
    enc_sent[data.type == 'ham'].sum(),
    enc_sent[data.type == 'spam'].sum()
])

In [47]:
word_probs = np.array([
    (enc_sent[data.type == 'ham'].sum(axis=0) + 1) / (total_words[0] + len(words)),
    (enc_sent[data.type == 'spam'].sum(axis=0) + 1) / (total_words[1] + len(words))
]).T

In [48]:
word_probs

array([[3.91686805e-02, 3.34983979e-03],
       [2.34702755e-02, 3.40809787e-02],
       [2.60608990e-02, 1.76958928e-02],
       ...,
       [1.93330111e-05, 1.01951646e-03],
       [3.86660222e-05, 9.46693854e-04],
       [2.70662156e-04, 7.28226041e-05]])

In [55]:
predict = np.array([np.argmax(class_prob * np.prod(word_probs[i], axis=0)) for i in enc_sent]).astype(bool)

In [53]:
np.mean((data.type == 'spam') == predict)

0.9865398420674802

In [56]:
np.mean((data.type == 'ham') == np.logical_not(predict))

0.9865398420674802

In [59]:
swords = words[np.argsort(word_probs[:, 1] - word_probs[:, 0])]

In [61]:
swords[-100:]

array(['attempt', 'word', 'collect', 'national', '800', '1st', '10', 'on',
       '10p', 'wk', '3', '2000', 'chance', 'selected', 'network', 'valid',
       'weekly', 'phone', 'every', 'this', 'mob', '5000', 'orange',
       'offer', 'collection', '000', 'receive', 'holiday', 'po', 'cost',
       'video', 'box', 'ringtone', 'rate', 'apply', 'shows', 'sms',
       'code', 'camera', 'min', 'landline', 'mins', 'line', 'latest',
       'chat', 'c', '150ppm', 'week', 'please', 'with', 'send', 'draw',
       'awarded', 'msg', '100', '1000', 'per', 'tone', '500', 'cs',
       'customer', 'nokia', 'guaranteed', 'com', 'only', '16', 'new',
       '18', 'co', 'contact', '50', 'service', 'win', 'urgent', 'a',
       'for', 'ur', 'cash', 'our', 'uk', 'won', '150p', '4', 'reply',
       'prize', '2', 'from', 'text', '1', 'stop', 'www', 'claim',
       'mobile', 'now', 'or', 'your', 'to', 'txt', 'free', 'call'],
      dtype='<U15')

In [66]:
print('\\t')

\t


In [65]:
print(r'\t')

\t


In [80]:
regex = re.compile(r'\w+ (\w+) (\w+)')

In [70]:
data.text[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [81]:
regex.findall(data.text[0])

[('until', 'jurong'),
 ('only', 'in'),
 ('n', 'great'),
 ('la', 'e'),
 ('there', 'got')]

In [83]:
re.match(regex, data.text[0]).group(0), re.match(regex, data.text[0]).group(1), re.match(regex, data.text[0]).group(2)

('Go until jurong', 'until', 'jurong')